# DB 연결 및 데이터 테스트

이 노트북은 Flask 애플리케이션의 데이터베이스에 연결하고 `RealEstateTransaction` 및 `PublicParking` 모델에서 데이터를 가져옵니다.

In [24]:
from utils import setup_db_context
import pandas as pd
from sqlalchemy import select

# DB 컨텍스트 설정
app, db = setup_db_context()

from myapp.models import RealEstateTransaction, PublicParking

데이터베이스 연결 및 앱 컨텍스트 푸시 완료.


## 1. 부동산 실거래가 (Real Estate Transactions)

In [25]:
# SQLAlchemy를 사용하여 데이터 가져오기
transactions = RealEstateTransaction.query.limit(5).all()
print(f"가져온 거래 수: {len(transactions)}")

# DataFrame으로 변환
stmt = select(RealEstateTransaction)
df_ret = pd.read_sql(stmt, db.session.connection())

print(f"총 레코드 수: {len(df_ret)}")
df_ret.head()

가져온 거래 수: 5
총 레코드 수: 235671


,ret_id,reception_year,district_code,district_name,legal_dong_code,legal_dong_name,jibun_type,jibun_type_name,main_number,sub_number,...,amount,building_area,land_area,floor,right_type,cancel_date,construction_year,building_use,declaration_type,broker_district_name
0,2,2025,11320,도봉구,10600,방학동,1,대지,0638,0010,...,14800,37.66,16.0,4,,,1994.0,연립다세대,중개거래,서울 도봉구
1,3,2025,11380,은평구,10800,역촌동,,,,,...,39620,74.08,182.0,,,,1972.0,단독다가구,직거래,
2,4,2025,11500,강서구,10200,등촌동,1,대지,0628,0013,...,10300,26.16,35.71,6,,,1999.0,오피스텔,중개거래,서울 강서구
3,5,2025,11620,관악구,10200,신림동,1,대지,1639,0051,...,12700,18.70,26.16,8,,,2014.0,오피스텔,중개거래,서울 관악구
4,6,2025,11530,구로구,10600,고척동,1,대지,0339,0000,...,84000,59.89,0.0,9,,,2009.0,아파트,중개거래,"서울 구로구, 서울 양천구"


In [ ]:
# SQLAlchemy를 사용하여 데이터 가져오기
transactions = RealEstateTransaction.query.limit(5).all()
print(f"가져온 거래 수: {len(transactions)}")

# DataFrame으로 변환
stmt = select(RealEstateTransaction)
df_ret = pd.read_sql(stmt, db.session.connection())

print(f"총 레코드 수: {len(df_ret)}")

# 전체 구 | 평단가 계산 | 건물 가격 단위(만원) | 건물 면적 단위 (m^2) | 모든 건물의 평균 단가 개발 계산 | price_per_sqm 컬럼 생성
def calc_price_per_sqm(df):
    df = df.copy()
    df['price_per_sqm'] = (
        df['amount'] * 10000
    ) / df['building_area']
    return df

def calc_yearly_avg_price_by_building(df):
    return (
        df
        .groupby(['building_use', 'reception_year'])
        .agg(
            avg_price_per_sqm=('price_per_sqm', 'mean'),
            transactions_counts=('price_per_sqm', 'count')
        )
        .reset_index()
        .sort_values(['building_use', 'reception_year'])
    )

def calc_yoy_change_rate_by_building(df):
    df = df.copy()

    df['yoy_change_rate'] = (
        df
        .groupby('building_use')['avg_price_per_sqm']
        .pct_change()
        .mul(100)
        .round(2)
    )

    return df

avg_df = calc_price_per_sqm(df_ret)
building_df = calc_yearly_avg_price_by_building(avg_df)
result = calc_yoy_change_rate_by_building(building_df)

result.fillna('-', inplace=True)

result

가져온 거래 수: 5
총 레코드 수: 235671


C:\Users\User\AppData\Local\Temp\ipykernel_17156\1931278703.py:49: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  result.fillna('-', inplace=True)


,building_use,reception_year,avg_price_per_sqm,transactions_counts,yoy_change_rate
0,단독다가구,2022,9.862544e+06,4892,-
1,단독다가구,2023,9.988841e+06,2753,1.28
2,단독다가구,2024,9.949955e+06,3339,-0.39
3,단독다가구,2025,1.244116e+07,110,25.04
4,아파트,2022,1.379078e+07,13030,-
5,아파트,2023,1.408201e+07,35634,2.11
6,아파트,2024,1.532464e+07,55773,8.82
7,아파트,2025,1.565109e+07,1657,2.13
8,연립다세대,2022,8.113295e+06,32941,-
9,연립다세대,2023,8.320594e+06,22670,2.56


## 2. 공영주차장 (Public Parkings)

In [28]:
# SQLAlchemy를 사용하여 데이터 가져오기
parkings = PublicParking.query.limit(5).all()
print(f"가져온 주차장 레코드 수: {len(parkings)}")

# DataFrame으로 변환
stmt = select(PublicParking)
df_pp = pd.read_sql(stmt, db.session.connection())

print(f"총 레코드 수: {len(df_pp)}")
df_pp.head()

가져온 주차장 레코드 수: 5
총 레코드 수: 1875


,pp_id,parking_code,parking_name,address,parking_type,parking_type_name,operation_type,operation_type_name,phone_number,parking_status_available,...,bus_basic_time_min,bus_add_rate,bus_add_time_min,day_max_rate,lat,lng,share_parking_company_name,share_parking,share_parking_company_link,share_parking_etc
0,1,171721,세종로 공영주차장(시),종로구 세종로 80-1,NW,노외 주차장,1,시간제 주차장,02-2290-6566,1,...,0,0,0,30900,37.573403,126.975884,,N,,
1,2,171730,종묘주차장 공영주차장(시),종로구 훈정동 2-0,NW,노외 주차장,1,시간제 주차장,02-2290-6166,1,...,0,0,0,28800,37.571504,126.994969,,N,,
2,3,171900,훈련원공원 공영주차장(시),중구 을지로5가 40-3,NW,노외 주차장,1,시간제 주차장,02-3405-4597,1,...,0,0,0,39600,37.567400,127.003521,,N,,
3,4,172051,한강진역 공영주차장(시),용산구 한남동 728-27,NW,노외 주차장,1,시간제 주차장,02-795-6406,1,...,0,0,0,28800,37.539522,127.002580,,N,,
4,5,172065,용산주차빌딩 공영주차장(시),용산구 한강로2가 12-9,NW,노외 주차장,1,시간제 주차장,02-2290-6014,1,...,0,0,0,21600,37.534364,126.965418,,N,,
